In [1]:
import os

In [2]:
%pwd

'/home/fintechsys/MLflow projects/kidney-Disease-classification-Deep-Learning--project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/fintechsys/MLflow projects/kidney-Disease-classification-Deep-Learning--project'

# Create MLflow URL 

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/eacomunication2014/kidney-Disease-classification-Deep-Learning--project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="eacomunication2014"
os.environ["MLFLOW_TRACKING_PASSWORD"]="84c1acca6bbe2c8430d655f20b92962a34f67535"

In [6]:
import tensorflow as tf

2024-02-17 23:41:08.560966: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 23:41:08.596811: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 23:41:08.597425: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-17 23:41:09.388176: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
model = tf.keras.models.load_model("artifacts/training/best_model.h5")

# Update entity

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

# Update configuration

In [11]:


from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/best_model.h5",
            training_data="artifacts/data_preprocessing",
            mlflow_uri="https://dagshub.com/eacomunication2014/kidney-Disease-classification-Deep-Learning--project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

# Evaluate model and log it to MLflow

In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [17]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode="binary"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

# Create the Pipeline

In [18]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-02-17 23:44:35,060: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-17 23:44:35,062: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-17 23:44:35,063: INFO: common: created directory at: artifacts]
Found 1370 images belonging to 2 classes.


2024-02-17 23:44:35.475449: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


86/86 [==============================] - 193s 2s/step - loss: 0.2072 - accuracy: 0.9302
[2024-02-17 23:47:48,492: INFO: common: json file saved at: scores.json]


2024/02/17 23:47:50 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-02-17 23:47:51,593: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /tmp/tmpxnj0fwgg/model/data/model/assets
[2024-02-17 23:47:51,986: INFO: builder_impl: Assets written to: /tmp/tmpxnj0fwgg/model/data/model/assets]


/home/fintechsys/anaconda3/envs/kidney/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2024-02-17 23:50:59,520: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /eacomunication2014/kidney-Disease-classification-Deep-Learning--project.mlflow/api/2.0/mlflow-artifacts/artifacts/4ec5f5f31ee148ee9469a64c16be3854/c46febcdf35d4ba5a6d4caa081791fa8/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2024-02-17 23:52:22,811: WARNING: connectionpool: Retrying (Retry(total=3, connect=5, read=3, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /eacomunication2014/kidney-Disease-classification-Deep-Learning--project.mlflow/api/2.0/mlflow-artifacts/artifacts/4ec5f5f31ee148ee9469a64c16be3854/c46febcdf35d4ba5a6d4caa081791fa8/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2024-02-17 23:59:23,324: WARNING: connectionpool: Retrying (Retry(tot

Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/02/18 00:31:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 23
Created version '23' of model 'VGG16Model'.
